In [1]:
!pip install selenium===4.21.0

  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 7.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 11.1 MB/s eta 0:00:0000:01
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)


In [120]:
import time
import urllib
import requests

# import BeautifulSoup from bs4
from bs4 import BeautifulSoup

In [121]:
def extract_pdf_urls(url, max_retries=3, delay=5):
    """
    Extracts PDF URLs from a given URL, handling redirects and retries.

    Args:
        url (str): The URL to fetch and extract URLs from.
        max_retries (int): The maximum number of retries.
        delay (int): The delay in seconds between retries.

    Returns:
        list: A list of extracted PDF URLs.
    """

    for attempt in range(max_retries):
        response = requests.get(url, allow_redirects=False)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, "html.parser")

            # Find all anchor tags with the desired href attribute
            pdf_links = []
            for link in soup.find_all("a", href=lambda href: href and "pdf=" in href):
                # Extract the content after 'pdf='
                pdf_link = link["href"].split("pdf=")[1]
                # Decode the URL-encoded string
                pdf_link = urllib.parse.unquote(pdf_link)
                # Replace the first '&' with '?'
                pdf_link = pdf_link.replace("&", "?", 1)
                pdf_links.append(pdf_link)

            return pdf_links

        elif response.status_code in [301, 302]:  # Check for redirect
            # Extract the actual URL from the Location header
            redirected_url = response.headers["Location"]
            print(f"Redirected from {url} to {redirected_url}")
            return extract_pdf_urls(redirected_url)  # Recursively call the function

        elif response.status_code == 502:
            print(f"Got a 502 error for {url}. Retrying in {delay} seconds...")
            time.sleep(delay)
        else:
            print(
                f"Failed to fetch the page. Status code: {response.status_code} for URL: {url}"
            )
            return []

    print(f"Max retries reached for {url}. Giving up.")
    return []

In [128]:
import requests
from bs4 import BeautifulSoup
import urllib.parse


def extract_pdf_urls(url):
    """
    Extracts PDF URLs from a given URL, handling redirects and using a browser-like user agent.

    Args:
        url (str): The URL to fetch and extract URLs from.

    Returns:
        list: A list of extracted PDF URLs.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers, allow_redirects=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Find all anchor tags with the desired href attribute
        pdf_links = []
        for link in soup.find_all("a", href=lambda href: href and "pdf=" in href):
            # Extract the content after 'pdf='
            pdf_link = link["href"].split("pdf=")[1]
            # Decode the URL-encoded string
            pdf_link = urllib.parse.unquote(pdf_link)
            # Replace the first '&' with '?'
            pdf_link = pdf_link.replace("&", "?", 1)
            pdf_links.append(pdf_link)

        return pdf_links

    else:
        print(
            f"Failed to fetch the page. Status code: {response.status_code} for URL: {url}"
        )
        return []

In [130]:
# Example usage:
url = "https://www2.trustnet.com/Factsheets/Factsheet.aspx?fundCode=J8F4M&univ=U"

# Extract PDF URLs
pdf_urls = extract_pdf_urls(url)

In [132]:
pdf_urls

['https://documentscdn.financialexpress.net/Literature/FF097F28977D792E9EF0B94E6C5C8D2A/214147324.pdf?citicode=J84N&universe=O',
 'https://www2.trustnet.com//Factsheets/FundFactsheetPDF.aspx?fundCode=J8F4M?univ=O&citicode=J84N&Universe=O&FundCode=J8F4M&SectorCode=O:VOM&citicode=J84N&universe=O']

In [133]:
import requests


def download_pdf(url, filename):
    """
    Downloads a PDF file from a given URL and saves it to a file.

    Args:
        url (str): The URL of the PDF file.
        filename (str): The desired filename for the downloaded PDF.
    """

    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {url} to {filename}")
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")

In [134]:
download_pdf(pdf_urls[0], "example.pdf")

Downloaded https://documentscdn.financialexpress.net/Literature/FF097F28977D792E9EF0B94E6C5C8D2A/214147324.pdf?citicode=J84N&universe=O to example.pdf


In [112]:
import pandas as pd


def process_funds_managed_column(csv_file):
    """
    Processes the "Funds managed" column in a CSV file, extracts individual URLs,
    downloads the HTML content, and extracts PDF URLs in parallel, using batch processing.

    Args:
        csv_file (str): The path to the CSV file.
        output_dir (str): The directory to save the downloaded HTML files.
        batch_size (int): The number of URLs to process in each batch.
    """

    df = pd.read_csv(csv_file)

    # Split "Funds managed" column into a list of URLs
    df["Funds managed"] = df["Funds managed"].str.split("https://")
    df["Funds managed"] = df["Funds managed"].apply(
        lambda x: [f"https://{url}" for url in x if url]
    )

    return df

In [113]:
# Example usage:
csv_file = "sara_funds/all_funds_1_1835.csv"  # Replace with your CSV file path
all_urls = process_funds_managed_column(csv_file)

In [111]:
all_urls

,AlphaRating,Property,Manager,Funds managed,Fund Focus,Files
0,NaN,NaN,https://www2.trustnet.com/managers/factsheet/-...,[https://www2.trustnet.com/Factsheets/Factshee...,Equity,NaN
1,NaN,NaN,https://www2.trustnet.com/managers/factsheet/2...,[https://www2.trustnet.com/Factsheets/Factshee...,Fixed Interest,NaN
2,NaN,NaN,https://www2.trustnet.com/managers/factsheet/7...,[https://www2.trustnet.com/Factsheets/Factshee...,Mixed AssetMixed AssetMixed AssetMixed AssetMi...,https://www2.trustnet.com/images/managerPerfWa...
3,NaN,NaN,https://www2.trustnet.com/managers/factsheet/a...,[https://www2.trustnet.com/Factsheets/Factshee...,Equity,https://www2.trustnet.com/images/managerPerfWa...
4,NaN,NaN,https://www2.trustnet.com/managers/factsheet/a...,[https://www2.trustnet.com/Factsheets/Factshee...,Fixed Interest,https://www2.trustnet.com/images/managerPerfWa...
...,...,...,...,...,...,...
1830,NaN,NaN,https://www2.trustnet.com/managers/factsheet/z...,[httpsJPM UK Equity CoreJPM UK Equity Plus],EquityEquity,NaN
1831,NaN,NaN,https://www2.trustnet.com/managers/factsheet/z...,[https://www2.trustnet.com/Factsheets/Factshee...,Equity,NaN
1832,NaN,NaN,https://www2.trustnet.com/managers/factsheet/z...,[httpsHL Multi Manager Special SituationsHL Mu...,EquityMixed AssetMixed AssetEquity,NaN
1833,NaN,NaN,https://www2.trustnet.com/managers/factsheet/z...,[httpsBNY Mellon Emerging IncomeBNY Mellon Asi...,EquityEquity,NaN
